# Recuperação de Portadora

Este notebook contém algumas implementações dos principais algoritmos que realizam a recuperação de portadora em receptores ópticos digitais coerentes, com a finalidade de compensar o deslocamento de frequência da portadora e o ruído de fase.

## 6.1 Frequency Recovery

Existem dois efeitos de fase principais que afetam os sistemas ópticos coerentes digitais. O primeiro é um deslocamento de frequência da portadora, o segundo é o ruído de fase.

Assumindo uma detecção coerente intradina, os lasers transmissores e osciladores locais não operam exatamente na mesma frequência, mas exibem um deslocamento que pode atingir a ordem de alguns gigahertz. Após o front-end óptico, esse deslocamento de frequência é traduzido para o domínio elétrico, levando a uma perda completa da referência de fase para o processo de decisão.

Assumindo uma equalização perfeita e demultiplexação de polarização, um sinal equalizado $y[k]$, prejudicado por um deslocamento de frequência, ruído de fase e ruído aditivo, pode ser expresso como:

$$ y[k]=s[k]e^{j(\theta[k]+k2\pi\Delta f T_{s})}+\eta[k]$$

Onde $s[k]$ é o sinal transmitido, $\theta[k]$ é o ruído de fase, $\Delta f$ é o deslocamento de frequência da portadora, $T_s$ é o período do símbolo e $\eta [k]$ é o ruído gaussiano branco aditivo (AWGN). Como o ruído de fase e os deslocamentos de frequência da portadora têm propriedades diferentes, eles são compensados ​​por algoritmos diferentes. Os algoritmos de recuperação de frequência estimam e compensam $\Delta f$ , enquanto os algoritmos de recuperação de fase estimam e compensam $\theta[k]$.

## 4th-Power Method

Um algoritmo eficaz de recuperação de frequência no domínio da frequência é baseado na análise espectral do sinal recebido elevado à 4ª potência. Este algoritmo foi originalmente proposto para ser utilizado em esquemas QPSK, mas também pode ser estendido para constelações M-QAM com uma perda tolerável de desempenho. 

Se os ruídos aditivos e de fase forem desprezados, e $s[k]$ tiver modulação QPSK, o sinal equalizado é dado por:

$$y[k]=e^{j({\frac{\pi}{4}}+m[k]{\frac{\pi}{2}})}e^{j k2\pi\Delta f T_{s}}$$ 

Onde $m[k] = 0, 1, 2, 3$ indica o quadrante do símbolo QPSK $s[k]$. Aumentar $y[k]$ para a 4ª potência produz:

$$(y[k])^{4}=e^{j\pi}e^{j k2\pi(4\Delta f)T_{s}}$$

Podemos observar pela equação acima que a operação de 4ª potência remove a dependência de $s[k]$ e gera um tom de frequência em $4\Delta f$ . O deslocamento de frequência pode então ser estimado calculando esse tom de frequência e dividindo-o por quatro. Isto não se aplica à modulação M-QAM com M > 4. No entanto, o espectro do sinal elevado à 4ª potência ainda tem um pico em $4\Delta f$, já que algumas partições das constelações M-QAM são constelações QPSK.

A estimativa do deslocamento de frequência, $\hat{\Delta f}$ é então obtido como:

$$\hat{\Delta f} = \frac{1}{4}\operatorname*{max}_{f}|FFT{(y[k])^4}|$$

onde $FFT\{·\}$ é a transformada rápida de Fourier e $f$ é o valor espectral máximo. Uma vez obtido $\hat{\Delta f}$, o deslocamento de frequência da portadora pode ser compensado como:

$$z[k] = y[k]e^{-jk2\pi\hat{\Delta f}T_s}$$

Como os mesmos lasers transmissor e oscilador local são usados ​​para gerar e detectar ambos os sinais multiplexados por polarização $s_1[k]$ e $s_2[k]$, os deslocamentos de frequência experimentados pelos sinais desmultiplexados por polarização $y1[k]$ e $y2[k]$ são semelhantes. Portanto, a estimativa de $\Delta f$ pode ser realizada em apenas um dos sinais ($y1[k]$ ou $y2[k]$), enquanto o resultado pode ser usado para corrigir ambos os sinais.

<br>
<center>
    <img src="https://i.postimg.cc/CLjBccvP/4th-Power-Method.png">
</center>
</br>